# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
session.execute('''

    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor':1}
    
''')

#### Set Keyspace

In [7]:
#Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkifydb')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# Based on the query we want to run, we use sessionid as the partition key to identify the partition and use the itemInSession as the  \ 
# clustering key to store row data in a sorted order. The sessionId and itemInSession are be used to locate one unique row.
query = 'CREATE TABLE IF NOT EXISTS songplay '
query = query + '''
                    (
                    sessionid INT,
                    itemInSession INT,
                    artist TEXT,
                    song TEXT,
                    length FLOAT,
                    PRIMARY KEY (sessionid, itemInSession)
                    )
    
                '''
session.execute(query)

In [9]:
# This a the csv file we created that contains all event data
file = 'event_datafile_new.csv'

# Loop the file and insert each row into the songplay table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = 'INSERT INTO songplay (sessionid, itemInSession, artist, song, length)'
        query = query + 'VALUES(%s, %s, %s, %s, %s)'
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
# Run the query to verify the data.
query = '''

SELECT 
    artist,
    song,
    length
FROM songplay
WHERE
    sessionId = 338
    AND itemInSession = 4

'''

rows = session.execute(query)
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# We will run the query based on userid and sessionid, so it's appropriate to use userid, sessionId as the partition key \
# and use the itemInSession as the clustering key. The three columns can be used to identify a unique row.
query = 'CREATE TABLE IF NOT EXISTS song_playlist_session '
query = query + '''
                    (
                    userId INT,
                    sessionId INT,
                    itemInSession INT,
                    song TEXT,
                    artist TEXT,
                    firstName TEXT,
                    lastName TEXT,
                    PRIMARY KEY ((userid, sessionId), itemInSession) 
                    )
    
                '''
session.execute(query)

# Loop the file and insert each row into the songplay table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = 'INSERT INTO song_playlist_session (userId, sessionId, itemInSession , song, artist, firstName, lastName)'
        query = query + 'VALUES(%s, %s, %s, %s, %s, %s, %s)'
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[9], line[0], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
# Run the query to verify the data.
query = '''

SELECT 
    itemInSession,
    artist,
    song,
    firstName,
    lastName
FROM song_playlist_session
WHERE
    userid = 10
    AND sessionid = 182

'''

rows = session.execute(query)
for row in rows:
    print(row.iteminsession, row.artist, row.song, row.firstname, row.lastname)

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [13]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Based on the query, we want user info based on a particular song. Therefore, the song column can be used as the partition key \
# and use the userId as the clustering key to store the data in a sorted order. The song and userId can be used to locate one unique row.
query = 'CREATE TABLE IF NOT EXISTS user_listen_ahaho '
query = query + '''
                    (
                    song TEXT,
                    userId INT,
                    firstName TEXT,
                    lastName TEXT,
                    PRIMARY KEY (song, userId)
                    )
    
                '''
session.execute(query)


# Loop the file and insert each row into the songplay table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = 'INSERT INTO user_listen_ahaho (song, userId, firstName, lastName)'
        query = query + 'VALUES(%s, %s, %s, %s)'
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
# Run the query to verify the data.
query = '''

SELECT 
    firstName,
    lastName
FROM user_listen_ahaho
WHERE
    song = 'All Hands Against His Own'
'''

rows = session.execute(query)
for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [15]:
## Drop the table before closing out the sessions
lst_tb = ['songplay', 'song_playlist_session', 'user_listen_ahaho']
query = 'DROP TABLE '

for tb in lst_tb:
    session.execute(query + tb)

### Close the session and cluster connection¶

In [16]:
session.shutdown()
cluster.shutdown()